In [1]:
import os
import json
import glob
import shutil
import cv2
from math import dist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']

In [13]:
result="../mmpose/k_fold/result"
ear_types = ["free", "attached"]


for ear_type in ear_types:
    names = os.listdir(os.path.join(result, ear_type))
    for name in names:
        df = pd.DataFrame()
        result_name_test_json = os.path.join(result, ear_type, name, "5_test_json")
        
        file = glob.glob(os.path.join(result_name_test_json, "*.json"))
        with open(file[0], "r") as f:
            gt_file = json.load(f)

        gt_keypoints = gt_file["annotations"]
        
        if ear_type == "free":
            for i in range(0, len(gt_keypoints)):
                kpts_pd = {}
                for j in range(0, 21):
                    kpts_pd[j] = str(gt_keypoints[i]["keypoints"][3*j])+","+str(gt_keypoints[i]["keypoints"][3*j+1])+","+str(gt_keypoints[i]["keypoints"][3*j+2])
                ser = pd.DataFrame(data=kpts_pd, index = [i])
                df = pd.concat([df, ser])
        else:
            for i in range(0, len(gt_keypoints)):
                kpts_pd = {}
                for j in range(0, 14):
                    kpts_pd[j] = str(gt_keypoints[i]["keypoints"][3*j])+","+str(gt_keypoints[i]["keypoints"][3*j+1])+","+str(gt_keypoints[i]["keypoints"][3*j+2])
                ser = pd.DataFrame(data=kpts_pd, index = [i])
                df = pd.concat([df, ser])

        count = 0
        for deg in degrees:
            after_inpainting_name_deg = os.path.join("../keypoint/{eartype}/4_after_inpainting".format(eartype = ear_type), name, deg)
            frame_count = len(os.listdir(after_inpainting_name_deg))

            if not os.path.isdir(os.path.join(result, ear_type, name, "ground_truth",deg)):
                os.makedirs(os.path.join(result, ear_type, name, "ground_truth",deg))
            df.iloc[count:count+frame_count,:].to_csv(os.path.join(result, ear_type, name, "ground_truth", deg, "gt.csv"))
            count = count + frame_count
    
    
    

../mmpose/k_fold/result\free\cather\5_test_json
../mmpose/k_fold/result\free\daniel\5_test_json
../mmpose/k_fold/result\free\dominic\5_test_json
../mmpose/k_fold/result\free\jack\5_test_json
../mmpose/k_fold/result\free\jakaria\5_test_json
../mmpose/k_fold/result\free\jimmy\5_test_json
../mmpose/k_fold/result\free\vicky\5_test_json
../mmpose/k_fold/result\free\wayne\5_test_json
../mmpose/k_fold/result\attached\gaby\5_test_json
../mmpose/k_fold/result\attached\matt\5_test_json
../mmpose/k_fold/result\attached\toby\5_test_json
../mmpose/k_fold/result\attached\wendy\5_test_json
../mmpose/k_fold/result\attached\willy\5_test_json


In [14]:
result="../mmpose/k_fold/result"
ear_types = ["free", "attached"]
# result="../7_result"
# final = "../7_final"
# cm = [4.72, 4.74, 3.9, 3.91, 4.20, 4.60, 4.69, 4.75]
# [4.05, 5.0, 4.26, 4.14, 4.51]

cm_index = 0
cm = {"cather":4.72, "daniel":4.74, "dominic":3.9, "jack":3.91, "jakaria":4.20, "jimmy":4.60, "vicky":4.69, "wayne":4.75,
      "gaby":4.05, "matt":5.0, "toby":4.26, "wendy":4.14, "willy":4.51}


for ear_type in ear_types:
    names = os.listdir(os.path.join(result, ear_type))
    for name in names:
        for deg in degrees:
            ground_truth_deg = os.path.join(result, ear_type, name, "ground_truth", deg)
            pred_deg = os.path.join(result, ear_type, name, "pred", deg)
            
            gt = glob.glob(os.path.join(ground_truth_deg, "*.csv"))[0]
            pred = glob.glob(os.path.join(pred_deg, "*.csv"))[0]

            gt_df = pd.read_csv(gt,index_col = 0)
            pred_df = pd.read_csv(pred, index_col = 0)

            error_df = pd.DataFrame(columns=gt_df.columns, index = gt_df.index)

            assert len(gt_df) == len(pred_df), "different frames"
            for i in range(0, len(gt_df)):

                scale0_x = float(gt_df.iloc[i,0].split(",")[0])
                scale0_y = float(gt_df.iloc[i,0].split(",")[1])

                scale11_x = float(gt_df.iloc[i,11].split(",")[0])
                scale11_y = float(gt_df.iloc[i,11].split(",")[1])

                scale = cm[name] / dist((scale0_x, scale0_y), (scale11_x, scale11_y))

                for j in range(0, len(gt_df.columns)):
                    if gt_df.iloc[i,j].split(",")[2] == "2":
                        pred_x = float(pred_df.iloc[i,j].split(",")[0])
                        pred_y = float(pred_df.iloc[i,j].split(",")[1])

                        gt_x = float(gt_df.iloc[i,j].split(",")[0])
                        gt_y = float(gt_df.iloc[i,j].split(",")[1])

                        error_df.iloc[i,j] = scale * dist((gt_x, gt_y), (pred_x, pred_y))



            result_name_error_deg = os.path.join(result, ear_type, name, "error",deg)
            if not os.path.isdir(result_name_error_deg):
                os.makedirs(result_name_error_deg)
            error_df.to_csv(os.path.join(result_name_error_deg, "error.csv"))

    
